# Documentation - Modèle de prédiction des Appels d'Offres
### Hartwood Consulting - Confidentiel


Aout 2018

***

Implementation Python d'un modèle prédictif permettant d'anticiper les Appels d'Offres (AO) des clients grands comptes.

**En entrée:**
    La totalité des AO reçus 
    
**En sortie:**
     Le nombre d'Appels d'Offres prévus pour la (ou les) prochaine(s) période(s)

A terme l'objectif étant d'avoir une prédiction fiable, répartie par sous-entité / métier / localisation, pour les 3 prochains mois.

# Configuration

### **Python 3.6.5 - 64 bits**
https://www.python.org/downloads/release/python-365/

### Environnement de développement / IDE
Pycharm Community (plus que) conseillé

https://www.jetbrains.com/pycharm/download/



### VENV - Environement Virtuel

VENV pour Virtual Environment

> La force de Python est ses milliers de packages, des bibliothèques qu'il suffit d'importer pour bénéficier de fonctions avancées. Pour éviter d'encombrer l'installation de Python de ses paquets, et permettre une meilleure portabilité, il est conseillé de travailler avec des environements virtuels.

Je vous conseille de [créer le votre en utilisant Pycharm](https://www.jetbrains.com/help/pycharm/creating-virtual-environment.html)

Ensuite, a partir du Terminal, lancez la commande suivante : 

&rarr;  pip install -r requirements.txt

Elle va installer tous les paquets nécessaires au bon fonctionnement du projet. Si tout c'est bien passé, vous êtes parés pour commencer.

Pour information, le fichier requirements.txt a été créé grâce à  [pipreqs](https://pypi.org/project/pipreqs/), qui permet d'exporter uniquement les paquets utilisé pour ce projet en question, et non tous les paquets installés dans le VENV.
___

### Jupyter Notebook

**Ce fichier a été créé avec Jupyter Notebook.** Pour le modifier et en reprendre des élements, vous devez installer Jupyter.

Plus d'informations ici : http://jupyter.org/install.html


&#9658; **Méthode 1 : Avec PIP**

Si vous avez déjà configuré votre projet Python avec un VENV, le plus rapide/simple est d'utiliser pip

Depuis la console python

&rarr; pip install jupyter

Puis 

&rarr; jupyter notebook
 
Cela va ouvrir une page dans votre navigateur, depuis laquelle vous pouvez créer, modifier et executer des Notebooks



&#9658; **Méthode 2 : Anaconda Navigator**

La deuxième méthode consiste à installer [Anaconda Navigator](https://docs.anaconda.com/anaconda/install/).
___

### Plotly

Il faut juste mettre ses identifiants: username & password, dans un fichier '.credentials' lui même situé dans un dossier '.plotly', dans votre dossier de base (Home Directory)

Infos ici: https://plot.ly/python/getting-started/

Par exemple, sur l'ordinateur de Mika Rakotomanana, le dossier de base est 'mrakotomanana'

* dossier > 'mrakotomanana'<br><t>
     * dossier > '.plotly'
        * fichier > '.credentials'
    
Contenu du fichier .credentials

{

    "username": "mrakotomana",

    "api_key": "H9i9Cx1kWGQHImlMHh7Y",
}

***Attention au point (.) au début dans '.plotly' & '.credentials'***

***L'username est bien 'mrakotomana'***

Pour changer l'api_key et en obtenir une nouvelle, allez ici > https://plot.ly/settings/api

![Screenshot montrant le fichier .credentials dans l'explorateur de fichiers](img/plotly_config.png)


# Vue d'ensemble du travail réalisé

4 fichiers...
Role de chaque etc 

Data_Loader

Data_Preparation 

Explication One hot encode et du problème qu'il résoud etc

Explication LSTM
Pq ce choix, objectif, en quoi elle sont judicieuse pour répondre au problème 


Présentation des résultats et ouverture sur le bon et sur le moins bon


Prise de recul, qu'est ce qu'il y a de bon a garder etc

### data_loader.py

Fichier dont le but est de charger notre fichier de données, de formatter les cellules qui contiennent des caractères non désirés.

### data_format_hot_encode.py



### data_preparation.py



### lstm_model.py

Script qui charge les données en utilisant data_preparation

Il initialise ensuite un modèle [Keras Sequential](https://keras.io/models/sequential/), comprenant un layer LSTM et des layers densément connectés (Dense layer). 

***Le contenu de ce script a été complètement réécrit, amélioré et intégré à la classe TransferLearner.***

Les [cellules LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) sont, par leur nature, très adaptées à la prédiction de Time Series. Leur mémoire interne leur permet de comprendre les tendances de fond. 

Voir un exemple ici : [Using LSTMs to forecast time-series](https://towardsdatascience.com/using-lstms-to-forecast-time-series-4ab688386b1f)

## Transfer Learning 

***Le problème :*** on s'addresse à un problème avec une Time Series (ou Série Temporelle), soit des données indexée par le temps. Sur le compte Orange, il y en a environ 2400 sur une période de 3 ans. Le problème étant qu'on cherche à prédire le nombre d'appels d'offres sur une période donnée (par mois), il nous faut donc regrouper les données par mois (ou demi-mois/deux semaines). Cela nous donne 37 mois ou 74 périodes de deux semaines. 

Sans parler du nombre de paramètres, c'est très peu de données pour entrainer un réseaux de neurones, bien que celui-ci soit un très bon candidat comme modèle.

***La solution :*** Entrainer le modèle sur un dataset qui ressemble au notre, mais qui est beaucoup plus consistant (plus d'exemples, plus de paramètres). 

Il suffit ensuite de re-entrainer ce modèle avec notre propre dataset, pour qu'il apprenne les particularité de celui-ci, et voila, on devrait maintenant obtenir de meilleures prédictions.  


[Plus d'informations](https://machinelearningmastery.com/transfer-learning-for-deep-learning/) 

### Les résultats

#### Pre Train
Le 1er training se déroule sur l'historique du CAC40 depuis Février 1990 (soit 170 mois ou 341 périodes de 2 semaines), 10 fois plus conséquent que notre historique d'Appels d'Offres. 
Il est composé de 6 features, et on cherche à prédire la dernière : le prix de cloture (Close)

Après entrainement, on obtient un [score](https://en.wikipedia.org/wiki/Coefficient_of_determination) qui oscille entre 0.88 et 0.94 (sur 1), donc très prometteur.

In [1]:
import pandas as pd 
import numpy as np 

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.tools as tls

In [2]:
df_CAC = pd.read_csv("DATA/CAC40_1990.csv")
date_column = "Date"

df_CAC[date_column] = pd.to_datetime(df_CAC[date_column], dayfirst=False, errors='coerce')
df_CAC = df_CAC.set_index(date_column)

print("Aperçu Dataset CAC40")
df_CAC.head()

Aperçu Dataset CAC40


,Open,High,Low,Adj Close,Volume,Close
Date,,,,,,
1990-02-28,1836.0,1975.0,1827.0,1972.0,0,1972.0
1990-03-31,1951.0,2141.0,1937.0,2046.0,0,2046.0
1990-04-30,2056.0,2141.0,2048.0,2107.0,0,2107.0
1990-05-31,2102.0,2102.0,1972.0,2035.0,0,2035.0
1990-06-30,2025.0,2046.0,1945.0,1965.0,0,1965.0


#### 2eme Training
On change ensuite la taille d'entrée du réseaux, pour qu'elle corresponde à notre dataset d'Appels d'Offres (2 features).
La colonne AOs est le nombre d'appels d'offres reçus pendant une période, toutes entité et tous métiers confondus.

La colonne Ones est un paramètre contenant que des 1. C'est un paramètre artificiel ajouté car l'apprentissage d'un réseau devient très difficile avec un seul paramètre.

In [3]:
df_CAC = pd.read_csv("DATA/AO_Orange_ones_SemiMonth_2.csv")
date_column = "Date"

df_CAC[date_column] = pd.to_datetime(df_CAC[date_column], dayfirst=False, errors='coerce')
df_CAC = df_CAC.set_index(date_column)

print("Aperçu Dataset Appels d'Offres Orange")
df_CAC.head()

Aperçu Dataset Appels d'Offres Orange


,Ones,AOs
Date,,
2015-03-15,1.0,20.0
2015-03-31,1.0,21.0
2015-04-15,1.0,36.0
2015-04-30,1.0,23.0
2015-05-15,1.0,17.0


#### Ci dessous, deux graphes montrant les résultats: les prédictions comparées aux vraies valeurs (ils sont issus de 2 entrainements distincts du modèle).

In [4]:
tls.embed("https://plot.ly/~mrakotomana/12")

In [5]:
tls.embed("https://plot.ly/~mrakotomana/16")

#### &#9658; Les résultats ont l'air prometteurs. Les deux dernières valeurs sont anormalement basses, même pour une période estivale. Le modèle n'a  pas su les anticiper car ce sont des tendances qu'il n'a jamais rencontré auparavant. 

### Axes d'amélioration

En l'état, l'algo permet de prédire de manière fiable les futurs Appels d'Offres de tous les clients grands comptes. Sans plus de détail sur les métiers et les sous-entités concernés, ni sur la localisation.

#### Pour aller plus loin 
- Compléter le dataset : Plus d'exemples, plus de paramètres (si possible non catégoriques) 

- 